## Content of the current notebook:
* Merging DataFrames with Pandas
* This notebook closely follows:
https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html

In [3]:
# download function for a plot
!wget 'https://raw.githubusercontent.com/ReDI-School/python-data-science/master/redi/ws_18/utils/display.py'
from display import display_side_by_side as _display_side_by_side
display_side_by_side = _display_side_by_side(display)


Redirecting output to ‘wget-log.1’.


In [0]:
import pandas as pd

## Part 0: Intro to merge

In [5]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

display_side_by_side(df1, df2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


#### Merge creates a single table out of two tables

Usually one collum in each table is used to connect the different rows. Here we use 'employee'.

In [6]:
df3 = pd.merge(df1, df2, on='employee')
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### Implicit detection of merge column

Pandas can implictly detect with column have the same name and merge on these. 
But is this really making the code easier to read?

In [7]:
display_side_by_side(pd.merge(df1, df2), pd.merge(df1, df2, on='employee'))

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


## Many-to-One
The Number of rows in both tables do not need to match. In the example below 'Engineering' is doublicated in the first table. In the second table the group column is unique. After merging on group we get as many rows as in the first table. Some data (the supervisior 'Guido' in this example) is doublicated.

In [8]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display_side_by_side(df3, df4, pd.merge(df3, df4, on='group'))

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


## Many-to-Many
It is also possible that there are doublicates in the column you merge on in both tables. The resulting table is then constructed from any combinations of the matching rows in both tables. 

In [9]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display_side_by_side(df1, df5, pd.merge(df1, df5, on='group'))

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


In [10]:
display_side_by_side(
    df1[df1.group == 'Engineering'], 
    df5[df5.group == 'Engineering'], 
    pd.merge(df1[df1.group == 'Engineering'], df5[df5.group == 'Engineering'], on='group')
)

,employee,group
1,Jake,Engineering
2,Lisa,Engineering


,group,skills
2,Engineering,coding
3,Engineering,linux


,employee,group,skills
0,Jake,Engineering,coding
1,Jake,Engineering,linux
2,Lisa,Engineering,coding
3,Lisa,Engineering,linux


#### Question:
`In table 1 you have 100 different products with 100 entries each. In table 2 we have the same 100 products with 200 different entries each. How big is the resulting table?`

In [11]:
(100 * 100) * 200

2000000

## Part 1: A kind request for the BI Team
```
Dear BI Team, 
​
we are considering to open pop up stores in our main markets. Can you give me an overview of the number of customer we have in each city?
​
Thanks a lot!
```

In [15]:
# load the datasets
orders_url = 'https://raw.githubusercontent.com/ReDI-School/python-data-science/master/datasets/reseller2/orders.csv'
product_details_url = 'https://raw.githubusercontent.com/ReDI-School/python-data-science/master/datasets/reseller2/product_details.csv'
product_details_url = 'https://raw.githubusercontent.com/ReDI-School/python-data-science/a7dfe1203fe02a68026e8669db3dd04e7021fd9b/datasets/reseller2/product_details.csv'
shipping_details_url = 'https://raw.githubusercontent.com/ReDI-School/python-data-science/master/datasets/reseller2/shipping_details.csv'
orders = pd.read_csv(orders_url, parse_dates=['datetime_ordered'])
product_details = pd.read_csv(product_details_url)
shipping_details = pd.read_csv(shipping_details_url)

display_side_by_side(orders.head(), product_details.head(), shipping_details.head())

,order_id,product_id,purchase_price,sales_price,customer_id,used_promo_code,datetime_ordered
0,177,9493,89.0,119.99,325,True,2018-06-27 17:20:50
1,255,3771,31.0,49.99,436,False,2018-06-17 04:54:24
2,893,6902,3.5,4.49,532,True,2018-07-22 21:05:31
3,571,8141,6.0,13.99,86,False,2018-07-10 16:55:21
4,376,7787,5.4,13.99,125,True,2018-07-28 06:11:38


,p_id,product_name,product_brand
0,2448,T-shirt,Reebok
1,5425,Shoes,Adidas
2,1254,T-shirt,Jack Wolfskin
3,7787,T-shirt,Adidas
4,3617,Socks,Reebok


,order_id,shipping_location,datetime_shipped
0,904,Zürich,2018-07-02 08:31:27
1,119,Cologne,2018-07-15 01:25:24
2,193,Frankfurt,2018-07-12 11:22:16
3,237,Hamburg,2018-07-09 09:23:51
4,836,Munich,2018-06-29 09:16:11


In [0]:
# merge the table 'orders' and 'shipping_details'

In [0]:
# count the number of customer per location - Tip: check nunique()

In [0]:
# plot the previous results

## Part 2: Another request for the BI Team
```
Dear BI Team, 
​
another urgent question. What are our 3 brands we generate most sales with? We really need to focus on our strengths!
​
You are my heros!
```

In [0]:
# merge the table 'orders' and 'product_details' (the product_is named differently in one of the tables)

In [0]:
# answer the request above

### Advanced
Can you visualize the total sales per city and product_name?. E.g. how many sales are of Nike in Berlin.

In [0]:
# your solution

# Part3: Inner, Outer, Left, Right Join
![alt text](https://www.dofactory.com/Images/sql-joins.png)

By default merge is doing an inner join. Try out what happens if you use an outer, left or right join.


In [12]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display_side_by_side(df6, df7)

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


In [0]:
# by default merge is doing an 'inner' join
display_side_by_side(pd.merge(df6, df7), pd.merge(df6, df7, how='inner'))

In [14]:
# try out a outer, left and right join with the data above
pd.merge(df6, df7 , how='outer').fillna('nothing')

,name,food,drink
0,Peter,fish,nothing
1,Paul,beans,nothing
2,Mary,bread,wine
3,Joseph,nothing,beer


In [0]:
# Which join makes most 'sense' in this case from your perspective? Can you somehow fill the missing values?

### Advanced

Check if there is a difference in the results of part 1 and part 2 if you use a different type of join type (outer, left, right). Which one makes most sense? Is there missing data you might need to take into account?

When you suspect missing data in a real live situation, you should  try to estimate how much data might be missing and communicate this along your actual analysis.

In [0]:
# your solution
outermer=pd.merge(orders, shipping_details, on='order_id' , how='outer')
merged_ordered_shipping=pd.merge(orders, shipping_details, on='order_id' )

In [20]:
outermer.

order_id             0
product_id           9
purchase_price       9
sales_price          9
customer_id          9
used_promo_code      9
datetime_ordered     9
shipping_location    0
datetime_shipped     0
dtype: int64